In [1]:
#import module
from collections import Counter
import numpy as np


In [2]:
# Build Node
class Node:
	def __init__(self, feature=None, value=None, results=None, true_branch=None, false_branch=None):
		self.feature = feature #Thuộc tính để phân chia
		self.value = value	 # Giá trị của thuộc tính phân chia
		self.results = results # Lưu label của class nếu nó là leave
		self.true_branch = true_branch # Rẽ sanh nhánh T-branch nếu giá trị đúng với thuộc tính
		self.false_branch = false_branch # Rẽ sang nhánh F-branch nếu giá trị sai với thuộc tính



In [3]:
'''Build function calculate entropy
Tính số lần xuất hiện của data bằng cách sử dụng np.bincount
Tính xác suất của từng phần tử và dùng công thức đã trình bày để tính entropy
'''

def entropy(data):
	dem = np.bincount(data)
	probabilities = dem / len(data)
	entropy = -np.sum([p * np.log2(p) for p in probabilities if p > 0])     #đảm bảo điều kiện số trong hàm log luôn dương
	return entropy


In [4]:
'''
Build split_data function
Dựa vào feature và value để split data
Giá trị <= thì qua nhánh true, giá trị > thì qua nhánh false
Trả về các subset gồm feature(true_X, false_X) và label(true_y, false_y)
'''

def split_data(X, y, feature, value):
	true_line = np.where(X[:, feature] <= value)[0]
	false_line = np.where(X[:, feature] > value)[0]
	true_X, true_y = X[true_line], y[true_line]
	false_X, false_y = X[false_line], y[false_line]
	return true_X, true_y, false_X, false_y


In [5]:
'''
Build tree function by ID3 algorithm
input là feature X và label y
Đầu tiên, kiểm tra các label có đồng nhất không, nếu có trả về leave có label tương ứng
Lặp qua các feature, tính entropy, tính information gain và chọn information gain lớn nhất làm tiêu chí split data
Đệ quy để xây dựng các true_branch và false_branch
output là leave được gắn label tương ứng
'''

def decision_tree(X, y):
	if len(set(y)) == 1:
		return Node(results=y[0])

	best_inforgain = 0
	tieu_chuan = None
	best_sets = None
	n_features = X.shape[1]

	current_entropy = entropy(y)

	for feature in range(n_features):
		feature_values = set(X[:, feature])
		for value in feature_values:
			true_X, true_y, false_X, false_y = split_data(X, y, feature, value)
			true_entropy = entropy(true_y)
			false_entropy = entropy(false_y)
			p = len(true_y) / len(y)
			inforgain = current_entropy - p * true_entropy - (1 - p) * false_entropy

			if inforgain > best_inforgain:
				best_inforgain = inforgain
				tieu_chuan = (feature, value)
				best_sets = (true_X, true_y, false_X, false_y)

	if best_inforgain > 0:
		T_branch = decision_tree(best_sets[0], best_sets[1])
		F_branch = decision_tree(best_sets[2], best_sets[3])
		return Node(feature = tieu_chuan[0], value = tieu_chuan[1], true_branch = T_branch, false_branch = F_branch)

	return Node(results=y[0])


In [6]:
'''
Build predict function
Đệ quy cho đến khi tìm thấy leave, trả về label ứng với nó
'''
def predict(tree, sample):
	if tree.results is not None:
		return tree.results
	else:
		branch = tree.false_branch
		if sample[tree.feature] <= tree.value:
			branch = tree.true_branch
		return predict(branch, sample)


In [7]:
#Testing
X = np.array([[2, 2], [2, 1], [2, 0], [1, 1],[1, 2], [1, 0], [0, 0], [0, 1], [0, 2]])
y = np.array([11, 32, 93, 4, 65, 26, 87, 48, 19])

# Building the tree
tree = decision_tree(X, y)

for sample in X:
    prediction = predict(tree, sample)
    print(f"Prediction for sample {sample}: {prediction}")



Prediction for sample [2 2]: 11
Prediction for sample [2 1]: 32
Prediction for sample [2 0]: 93
Prediction for sample [1 1]: 4
Prediction for sample [1 2]: 65
Prediction for sample [1 0]: 26
Prediction for sample [0 0]: 87
Prediction for sample [0 1]: 48
Prediction for sample [0 2]: 19
